# Deep Learning Model
## Luis Arturo
### A01703572

# I HATE TENSORFLOW >:V

# Conexión a GPU local

In [59]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import setuptools
import tensorflow as tf
import pandas as pd
import os

In [60]:
def verify_gpu_setup():
    """Comprehensive verification of TensorFlow GPU setup"""
    # Print TensorFlow version
    print(f"TensorFlow version: {tf.__version__}")
    
    # Check CUDA availability
    print("\nCUDA available:", tf.test.is_built_with_cuda())
    
    # List all available GPUs
    physical_devices = tf.config.list_physical_devices()
    print("\nAvailable devices:")
    for device in physical_devices:
        print(f"  {device.device_type}: {device.name}")
    
    # Check if GPU is available and perform a simple computation
    if tf.test.is_built_with_cuda():
        try:
            # Create some random data
            with tf.device('/GPU:0'):
                a = tf.random.normal([1000, 1000])
                b = tf.random.normal([1000, 1000])
                # Perform matrix multiplication
                c = tf.matmul(a, b)
                # Force execution
                result = c.numpy()
                print("\n✅ Successfully performed computation on GPU!")
                print(f"Matrix multiplication shape: {result.shape}")
        except Exception as e:
            print("\n❌ Error during GPU computation:")
            print(e)
    else:
        print("\n❌ CUDA is not available in this TensorFlow installation")

    # Print GPU memory info
    try:
        gpu = tf.config.list_physical_devices('GPU')[0]
        print("\nGPU Device:", gpu)
    except IndexError:
        print("\n❌ No GPU devices found")

verify_gpu_setup()

TensorFlow version: 2.18.0

CUDA available: True

Available devices:
  CPU: /physical_device:CPU:0
  GPU: /physical_device:GPU:0

✅ Successfully performed computation on GPU!
Matrix multiplication shape: (1000, 1000)

GPU Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [61]:
# Set TF logging level to avoid unnecessary messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ETL

## Carga de datos

In [62]:
# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [63]:
# 1. EXTRACT - Carga de datos
print("=== CARGANDO DATASETS ===")
games_df = pd.read_csv('data/games.csv')
users_df = pd.read_csv('data/users.csv')
recommendations_df = pd.read_csv('data/recommendations.csv')

print("=== DATASETS CARGADOS ===")

=== CARGANDO DATASETS ===
=== DATASETS CARGADOS ===


## Información general del dataset

In [64]:
print("\nDimensiones del games_df:", games_df.shape)
print("\nColumnas disponibles del games_df:")
for col in games_df.columns:
    print(f"- {col}")


Dimensiones del games_df: (50872, 13)

Columnas disponibles del games_df:
- app_id
- title
- date_release
- win
- mac
- linux
- rating
- positive_ratio
- user_reviews
- price_final
- price_original
- discount
- steam_deck


In [65]:
print("\nDimensiones del users_df:", users_df.shape)
print("\nColumnas disponibles del users_df:")
for col in users_df.columns:
    print(f"- {col}")


Dimensiones del users_df: (14306064, 3)

Columnas disponibles del users_df:
- user_id
- products
- reviews


In [66]:
print("\nDimensiones del recommendations_df:", recommendations_df.shape)
print("\nColumnas disponibles del recommendations_df:")
for col in recommendations_df.columns:
    print(f"- {col}")


Dimensiones del recommendations_df: (41154794, 8)

Columnas disponibles del recommendations_df:
- app_id
- helpful
- funny
- date
- is_recommended
- hours
- user_id
- review_id


### Análisis inicial de games

In [67]:
print("\nPrimeras 5 filas de games_df:")
print(games_df.head())
print("\nInformación del dataset de juegos:")
print(games_df.info())
print("\nEstadísticas descriptivas de games_df:")
print(games_df.describe())
print("\nValores nulos en games_df:")
print(games_df.isnull().sum())


Primeras 5 filas de games_df:
   app_id                              title date_release   win    mac  linux  \
0   13500  Prince of Persia: Warrior Within™   2008-11-21  True  False  False   
1   22364            BRINK: Agents of Change   2011-08-03  True  False  False   
2  113020       Monaco: What's Yours Is Mine   2013-04-24  True   True   True   
3  226560                 Escape Dead Island   2014-11-18  True  False  False   
4  249050            Dungeon of the ENDLESS™   2014-10-27  True   True  False   

          rating  positive_ratio  user_reviews  price_final  price_original  \
0  Very Positive              84          2199         9.99            9.99   
1       Positive              85            21         2.99            2.99   
2  Very Positive              92          3722        14.99           14.99   
3          Mixed              61           873        14.99           14.99   
4  Very Positive              88          8784        11.99           11.99   

   disc

### Análisis inicial de users

In [68]:
print("\nPrimeras 5 filas de users_df:")
print(users_df.head())
print("\nInformación del dataset de usuarios:")
print(users_df.info())
print("\nEstadísticas descriptivas de users_df:")
print(users_df.describe())
print("\nValores nulos en users_df:")
print(users_df.isnull().sum())


Primeras 5 filas de users_df:
    user_id  products  reviews
0   7360263       359        0
1  14020781       156        1
2   8762579       329        4
3   4820647       176        4
4   5167327        98        2

Información del dataset de usuarios:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14306064 entries, 0 to 14306063
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   products  int64
 2   reviews   int64
dtypes: int64(3)
memory usage: 327.4 MB
None

Estadísticas descriptivas de users_df:
            user_id      products       reviews
count  1.430606e+07  1.430606e+07  1.430606e+07
mean   7.153032e+06  1.163734e+02  2.876738e+00
std    4.129805e+06  2.438515e+02  7.987421e+00
min    0.000000e+00  0.000000e+00  0.000000e+00
25%    3.576516e+06  2.300000e+01  1.000000e+00
50%    7.153032e+06  5.500000e+01  1.000000e+00
75%    1.072955e+07  1.270000e+02  3.000000e+00
max    1.430606e+07  3.221400e+04  6.045000e+03

Valores 

### Análisis inicial de recommendations

In [69]:
print("\nPrimeras 5 filas de recommendations_df:")
print(recommendations_df.head())
print("\nInformación del dataset de recomendaciones:")
print(recommendations_df.info())
print("\nEstadísticas descriptivas de recommendations_df:")
print(recommendations_df.describe())
print("\nValores nulos en recommendations_df:")
print(recommendations_df.isnull().sum())


Primeras 5 filas de recommendations_df:
    app_id  helpful  funny        date  is_recommended  hours  user_id  \
0   975370        0      0  2022-12-12            True   36.3    51580   
1   304390        4      0  2017-02-17           False   11.5     2586   
2  1085660        2      0  2019-11-17            True  336.5   253880   
3   703080        0      0  2022-09-23            True   27.4   259432   
4   526870        0      0  2021-01-10            True    7.9    23869   

   review_id  
0          0  
1          1  
2          2  
3          3  
4          4  

Información del dataset de recomendaciones:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41154794 entries, 0 to 41154793
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   app_id          int64  
 1   helpful         int64  
 2   funny           int64  
 3   date            object 
 4   is_recommended  bool   
 5   hours           float64
 6   user_id         int64  
 7  

### Análisis expecífico de features importantes

In [70]:
# Análisis de ratings
print("\nDistribución de ratings en juegos:")
print(games_df['rating'].value_counts())


Distribución de ratings en juegos:
rating
Positive                   13502
Very Positive              13139
Mixed                      12157
Mostly Positive             8738
Mostly Negative             1849
Overwhelmingly Positive     1110
Negative                     303
Very Negative                 60
Overwhelmingly Negative       14
Name: count, dtype: int64


In [71]:
# Análisis de precios
print("\nEstadísticas de precios:")
print(games_df[['price_final', 'price_original']].describe())


Estadísticas de precios:
        price_final  price_original
count  50872.000000    50872.000000
mean       8.620325        8.726788
std       11.514164       11.507021
min        0.000000        0.000000
25%        0.990000        0.990000
50%        4.990000        4.990000
75%       10.990000       11.990000
max      299.990000      299.990000


In [72]:
# Análisis de recomendaciones
print("\nDistribución de recomendaciones:")
print(recommendations_df['is_recommended'].value_counts(normalize=True))


Distribución de recomendaciones:
is_recommended
True     0.857844
False    0.142156
Name: proportion, dtype: float64


In [73]:
# Análisis de horas jugadas
print("\nEstadísticas de horas jugadas:")
print(recommendations_df['hours'].describe())


Estadísticas de horas jugadas:
count    4.115479e+07
mean     1.006022e+02
std      1.761675e+02
min      0.000000e+00
25%      7.800000e+00
50%      2.730000e+01
75%      9.920000e+01
max      1.000000e+03
Name: hours, dtype: float64


### Verificamos la integridad de referencias entre los datasets
Nos aseguramos de que los juegos y los usuarios referenciados en reseñas existen en el dataset de juegos y en el de usuarios

In [74]:
# Verificar si todos los app_id en recommendations existen en games
games_apps = set(games_df['app_id'])
recommendations_apps = set(recommendations_df['app_id'])
print("\nJuegos en recommendations pero no en games:", len(recommendations_apps - games_apps))


Juegos en recommendations pero no en games: 0


In [75]:
# Verificar si todos los user_id en recommendations existen en users
users_ids = set(users_df['user_id'])
recommendations_users = set(recommendations_df['user_id'])
print("Usuarios en recommendations pero no en users:", len(recommendations_users - users_ids))

Usuarios en recommendations pero no en users: 0


## Se va a crear un modelo capaz de predecir el rating que tendrá un juego ("Very Positive", "Mixed", etc.) 
## Son 9 clases 
- Positive                   13502
- Very Positive              13139
- Mixed                      12157
- Mostly Positive             8738
- Mostly Negative             1849
- Overwhelmingly Positive     1110
- Negative                     303
- Very Negative                 60
- Overwhelmingly Negative       14

## Features de entrada
- Precio
- Plataformas soportadas (win, mac, linux)
- Tiempo en el mercado
- Descuentos
- Métricas agregadas de reviews (horas promedio jugadas, ratio de recomendaciones)

## Preprocesamiendo de los datos

In [76]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

### Creamos features en base a las recomendaciones de los juegos

In [77]:
game_metrics = recommendations_df.groupby('app_id').agg({
    'hours': ['mean', 'median', 'std'],
    'is_recommended': 'mean',
    'helpful': 'mean',
    'funny': 'mean'
}).reset_index()

# Aplanar los nombres de las columnas
game_metrics.columns = ['app_id', 'avg_hours', 'median_hours', 'std_hours', 
                        'recommendation_ratio', 'avg_helpful', 'avg_funny']

### Procesamos las fechas para calcular el tiempo que el juego ha estado en el mercado

In [78]:
games_df['date_release'] = pd.to_datetime(games_df['date_release'])
reference_date = pd.to_datetime('2024-11-01')  # Fecha de referencia
games_df['days_in_market'] = (reference_date - games_df['date_release']).dt.days

### Unimos los nuevos features en un dataset final usando como base el de games

In [79]:
final_df = games_df.merge(game_metrics, on='app_id', how='left')
    
# 4. Rellenar valores nulos
final_df = final_df.fillna({
    'avg_hours': 0,
    'median_hours': 0,
    'std_hours': 0,
    'recommendation_ratio': 0.5,
    'avg_helpful': 0,
    'avg_funny': 0
})

In [80]:
final_df.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,days_in_market,avg_hours,median_hours,std_hours,recommendation_ratio,avg_helpful,avg_funny
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True,5824,18.967912,12.9,41.722269,0.845789,5.392052,0.612100
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True,4839,0.000000,0.0,0.000000,0.500000,0.000000,0.000000
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True,4209,20.413294,6.9,53.295053,0.908541,1.434682,0.450382
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True,3636,10.776625,8.1,15.421851,0.625998,6.778791,0.815279
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True,3658,40.621691,23.4,54.860085,0.885567,2.530928,0.698351


In [81]:
final_df.describe()

,app_id,date_release,positive_ratio,user_reviews,price_final,price_original,discount,days_in_market,avg_hours,median_hours,std_hours,recommendation_ratio,avg_helpful,avg_funny
count,5.087200e+04,50872,50872.000000,5.087200e+04,50872.000000,50872.000000,50872.000000,50872.000000,50872.000000,50872.000000,50872.000000,50872.000000,50872.000000,50872.000000
mean,1.055224e+06,2019-03-13 03:53:57.112753664,77.052033,1.824425e+03,8.620325,8.726788,5.592212,2059.837533,12.017546,5.630656,21.209406,0.697721,2.927259,0.496619
min,1.000000e+01,1997-06-30 00:00:00,0.000000,1.000000e+01,0.000000,0.000000,0.000000,374.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.287375e+05,2017-03-21 00:00:00,67.000000,1.900000e+01,0.990000,0.990000,0.000000,1137.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000
50%,9.860850e+05,2019-08-23 00:00:00,81.000000,4.900000e+01,4.990000,4.990000,0.000000,1897.000000,3.436826,1.500000,4.186043,0.714286,2.178243,0.131972
75%,1.524895e+06,2021-09-21 00:00:00,91.000000,2.060000e+02,10.990000,11.990000,0.000000,2782.000000,10.730804,5.000000,21.601544,0.881546,4.000000,0.482362
max,2.599300e+06,2023-10-24 00:00:00,100.000000,7.494460e+06,299.990000,299.990000,90.000000,9986.000000,487.000000,511.500000,495.120177,1.000000,268.285714,266.285714
std,6.103249e+05,NaN,18.253592,4.007352e+04,11.514164,11.507021,18.606679,1125.133076,28.005183,16.127001,41.136308,0.204218,4.689921,2.481174


In [82]:
final_df.max()

app_id                              2599300
title                      🧠 OUT OF THE BOX
date_release            2023-10-24 00:00:00
win                                    True
mac                                    True
linux                                  True
rating                        Very Positive
positive_ratio                          100
user_reviews                        7494460
price_final                          299.99
price_original                       299.99
discount                               90.0
steam_deck                             True
days_in_market                         9986
avg_hours                             487.0
median_hours                          511.5
std_hours                        495.120177
recommendation_ratio                    1.0
avg_helpful                      268.285714
avg_funny                        266.285714
dtype: object

### Se encontró un juego que posee métricas exageradas, al compararlo con Steam nos dimos cuenta de que parece ser un error
En base a esto imprimimos todos los juegos que tienen un precion final mayor a 70

In [83]:
juegos_filtrados = final_df[final_df['price_final'] > 70]

In [84]:
juegos_filtrados

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,days_in_market,avg_hours,median_hours,std_hours,recommendation_ratio,avg_helpful,avg_funny
81,438450,3DF Zephyr Lite Steam Edition,2016-02-02,True,False,False,Very Positive,96,58,199.99,199.99,0.0,True,3195,243.356250,121.35,265.982864,1.000000,4.979167,0.750000
144,537770,Gal*Gun: Double Peace - 'Pheromone Z' Item,2016-10-20,True,False,False,Positive,91,12,89.99,89.99,0.0,True,2934,0.000000,0.00,0.000000,0.500000,0.000000,0.000000
190,554820,VideoPad Video Editor,2016-12-01,True,True,False,Very Positive,80,51,99.99,99.99,0.0,True,2892,221.103030,130.70,238.796225,0.787879,3.212121,0.242424
403,1076160,Command: Modern Operations,2019-11-14,True,False,False,Very Positive,84,897,79.99,79.99,0.0,True,1814,126.815050,49.00,186.176124,0.859532,12.749164,5.050167
675,1182920,Movavi Video Editor Plus 2020 - Video Editing ...,2019-11-25,True,True,False,Very Positive,80,874,74.99,74.99,0.0,True,1803,111.442762,56.80,161.577714,0.788419,1.817372,0.227171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47765,411893,DCS: F-14A/B Tomcat,2019-03-13,True,False,False,Very Positive,92,555,79.99,79.99,0.0,True,2060,0.000000,0.00,0.000000,0.500000,0.000000,0.000000
48354,1096900,RPG Maker MZ,2020-08-27,True,True,False,Very Positive,83,1070,80.00,0.00,0.0,True,1527,186.038028,72.85,249.288267,0.791080,16.969484,3.521127
49167,2199970,Substance 3D Painter 2023,2023-01-23,True,True,True,Very Positive,85,148,150.00,0.00,0.0,True,648,0.000000,0.00,0.000000,0.500000,0.000000,0.000000
49310,2070990,VEGAS Edit 20 Steam Edition,2022-11-01,True,False,False,Mixed,66,21,129.48,249.00,48.0,True,731,68.566667,5.60,113.331475,0.666667,19.333333,5.333333


Al ser solo 150 juegos, decidimos tirarlos para evitar conflictor

In [85]:
final_df = final_df[final_df['price_final'] <= 70].reset_index(drop=True)


In [86]:
final_df = final_df[final_df['price_original'] <= 70].reset_index(drop=True)


In [87]:
final_df.max()

app_id                              2599300
title                      🧠 OUT OF THE BOX
date_release            2023-10-24 00:00:00
win                                    True
mac                                    True
linux                                  True
rating                        Very Positive
positive_ratio                          100
user_reviews                        7494460
price_final                            70.0
price_original                        69.99
discount                               90.0
steam_deck                             True
days_in_market                         9986
avg_hours                             487.0
median_hours                          511.5
std_hours                        495.120177
recommendation_ratio                    1.0
avg_helpful                      268.285714
avg_funny                        266.285714
dtype: object

### Parece ser que hay juegos con nombres repetidos, eliminamos aquellos que esten repetidos ya que solo son 121 títulos problemáticos

In [88]:
titulos_duplicados = final_df['title'].duplicated().sum()
print(f"Número de títulos duplicados: {titulos_duplicados}")
titulos_repetidos = final_df[final_df['title'].duplicated(keep=False)]['title'].unique()
print(f"Títulos duplicados: {titulos_repetidos}")

Número de títulos duplicados: 121
Títulos duplicados: ['Prison Wars' 'The Good Life' 'RUSH' 'Hide and Seek' 'Remnants'
 'Blade of Darkness' 'The Cleaner' 'Rogue' 'Lighthouse Keeper' 'Valor'
 'Last Stop' 'fishy' 'Flashback' 'The Hotel' '3D Organon Anatomy'
 'Minotaur' 'Northern Lights' 'First Snow' 'Momentum' 'Alter Ego'
 'The Backrooms' 'Locked Up' 'Get Stuffed!' 'Ascent' 'The Hunt' 'Hatch'
 'Lost' 'Warhammer Quest' 'Apollo 11 VR' 'Psych' 'Eternal Return'
 'Achievement Clicker' 'Call of Duty®' 'Causality' 'The Lost Village'
 'Resonance' 'Dead Forest' 'Lost Marbles' 'Wanderer' 'Castles' 'Grapple'
 'Chaos Theory' 'Dungeon Warriors' 'Bounce' 'Evolution' 'Zombie Survivors'
 'Ritual' 'Archery Simulator' 'Outpost' 'Shutter' 'The Line' 'The Village'
 'The Wanderer' 'Fantasy Gladiators' 'Cave Crawler' 'Cursed'
 'A Walk in the Woods' 'Dark Matter' 'Beyond the Wall' 'White Mirror'
 'Zombie Apocalypse' 'RIFT' 'STAY' 'Silent World' 'REALITY' 'Tomorrow'
 'Arachnophobia' 'Vaccine' 'Arena' 'Dog Adven

In [89]:
titulos_unicos = ~final_df['title'].duplicated(keep=False)

# Filtrar el DataFrame para conservar solo los títulos únicos
final_df = final_df[titulos_unicos].reset_index(drop=True)
final_df.describe()

,app_id,date_release,positive_ratio,user_reviews,price_final,price_original,discount,days_in_market,avg_hours,median_hours,std_hours,recommendation_ratio,avg_helpful,avg_funny
count,5.047200e+04,50472,50472.000000,5.047200e+04,50472.000000,50472.000000,50472.000000,50472.000000,50472.000000,50472.000000,50472.000000,50472.000000,50472.000000,50472.000000
mean,1.053253e+06,2019-03-10 18:46:25.164051456,77.107149,1.827487e+03,8.334297,8.418423,5.593557,2062.217764,11.961956,5.609096,21.154918,0.698100,2.921674,0.494605
min,1.000000e+01,1997-06-30 00:00:00,0.000000,1.000000e+01,0.000000,0.000000,0.000000,374.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.279475e+05,2017-03-17 00:00:00,67.000000,1.900000e+01,0.990000,0.990000,0.000000,1141.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000
50%,9.840950e+05,2019-08-20 00:00:00,81.000000,4.900000e+01,4.990000,4.990000,0.000000,1900.000000,3.460451,1.500000,4.203173,0.716655,2.176318,0.131313
75%,1.521705e+06,2021-09-17 00:00:00,91.000000,2.080000e+02,10.990000,11.990000,0.000000,2786.000000,10.753019,5.000000,21.654140,0.881903,4.000000,0.480000
max,2.599300e+06,2023-10-24 00:00:00,100.000000,7.494460e+06,70.000000,69.990000,90.000000,9986.000000,487.000000,511.500000,495.120177,1.000000,268.285714,266.285714
std,6.095610e+05,NaN,18.217559,4.018612e+04,9.882176,9.686554,18.599731,1125.329458,27.762187,16.033585,40.894435,0.204009,4.688425,2.486727


In [90]:
titulos_duplicados = final_df['title'].duplicated().sum()
print(f"Número de títulos duplicados: {titulos_duplicados}")
titulos_repetidos = final_df[final_df['title'].duplicated(keep=False)]['title'].unique()
print(f"Títulos duplicados: {titulos_repetidos}")

Número de títulos duplicados: 0
Títulos duplicados: []


### Definimos las features que se usarán para el modelo

In [91]:
features = [
    'price_final', 'price_original', 'discount',
    'win', 'mac', 'linux', 'steam_deck',
    'days_in_market',
    'avg_hours', 'median_hours', 'std_hours',
    'recommendation_ratio', 'avg_helpful', 'avg_funny'
]

### Por los valores tan diferentes que tenemos en el dataset, normalizamos los datos para que puedan ser usados por el modelo sin problemas y no haya valores extremos que hagan que los valores menores no tengan importancia

In [92]:
scaler = StandardScaler()
numeric_features = [
    'price_final', 'price_original', 'discount',
    'days_in_market', 'avg_hours', 'median_hours', 'std_hours',
    'recommendation_ratio', 'avg_helpful', 'avg_funny'
]
final_df[numeric_features] = scaler.fit_transform(final_df[numeric_features])

### Pasamos la compatibilidad con diferentes plataformas a que sean booleanas, 0 o 1

In [93]:
bool_features = ['win', 'mac', 'linux', 'steam_deck']
final_df[bool_features] = final_df[bool_features].astype(int)

### Preparamos la variable dependiente (y)

In [94]:
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(final_df['rating'])
y = to_categorical(y)  # One-hot encoding para clasificación multiclase

### Preparamos las features (x)

In [95]:
X = final_df[features].values

# Embeddings

### Limpiamos los títulos

In [96]:
def clean_title(title):
    """
    Limpia y normaliza el título del juego.
    
    Args:
        title (str): Título original del juego
        
    Returns:
        str: Título limpio y normalizado
    """
    # Convertir a minúsculas
    title = title.lower()
    
    # Eliminar caracteres especiales y símbolos, manteniendo espacios
    title = re.sub(r'[^\w\s]', ' ', title)
    
    # Eliminar espacios múltiples
    title = ' '.join(title.split())
    
    return title

final_df['clean_title'] = final_df['title'].apply(clean_title)

### Creamos y configuramos el tokenizer

In [97]:
vocab_size = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(final_df['clean_title'])

### Convertimos los títulos a secuencias numéricas

In [98]:
title_sequences = tokenizer.texts_to_sequences(final_df['clean_title'])

### Determinamos el padding según la secuencia máxima que poseemos

In [99]:
title_lengths = [len(x) for x in title_sequences]
max_length = max(title_lengths)
avg_length = sum(title_lengths) / len(title_lengths)

print(f"\nEstadísticas de longitud de títulos:")
print(f"- Longitud máxima: {max_length} palabras")
print(f"- Longitud promedio: {avg_length:.2f} palabras")


Estadísticas de longitud de títulos:
- Longitud máxima: 28 palabras
- Longitud promedio: 3.56 palabras


### Hacemos padding de las secuencias

In [100]:
max_length = min(max_length, 10)  # Limitamos a 10 palabras como máximo
padded_sequences = pad_sequences(title_sequences, maxlen=max_length, padding='post')

### Ejemplo de procesamiento

In [101]:
ejemplo_titulo = final_df['title'].iloc[0]
ejemplo_limpio = final_df['clean_title'].iloc[0]
ejemplo_seq = title_sequences[0]
ejemplo_padded = padded_sequences[0]

print(f"Título original: {ejemplo_titulo}")
print(f"Título limpio: {ejemplo_limpio}")
print(f"Secuencia: {ejemplo_seq}")
print(f"Secuencia con padding: {ejemplo_padded}")

Título original: Prince of Persia: Warrior Within™
Título limpio: prince of persia warrior within
Secuencia: [1277, 3, 2537, 244, 659]
Secuencia con padding: [1277    3 2537  244  659    0    0    0    0    0]


In [102]:
print(f"\nTamaño del vocabulario: {len(tokenizer.word_index)}")


Tamaño del vocabulario: 29812


In [103]:
final_df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,days_in_market,avg_hours,median_hours,std_hours,recommendation_ratio,avg_helpful,avg_funny,clean_title
0,13500,Prince of Persia: Warrior Within™,2008-11-21,1,0,0,Very Positive,84,2199,0.167546,0.162245,-0.300736,1,3.342861,0.252359,0.454731,0.502943,0.723937,0.526915,0.047249,prince of persia warrior within
1,22364,BRINK: Agents of Change,2011-08-03,1,0,0,Positive,85,21,-0.540807,-0.560414,-0.300736,1,2.467553,-0.430877,-0.349838,-0.517311,-0.971047,-0.623174,-0.198900,brink agents of change
2,113020,Monaco: What's Yours Is Mine,2013-04-24,1,1,1,Very Positive,92,3722,0.673512,0.678429,-0.300736,1,1.907711,0.304422,0.080513,0.785937,1.031535,-0.317165,-0.017784,monaco what s yours is mine
3,226560,Escape Dead Island,2014-11-18,1,0,0,Mixed,61,873,0.673512,0.678429,-0.300736,1,1.398522,-0.042696,0.155357,-0.140193,-0.353433,0.822697,0.128956,escape dead island
4,249050,Dungeon of the ENDLESS™,2014-10-27,1,1,0,Very Positive,88,8784,0.369933,0.368719,-0.300736,1,1.418072,1.032340,1.109613,0.824208,0.918921,-0.083344,0.081934,dungeon of the endless
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50467,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,1,0,0,Very Positive,96,101,1.382877,-0.869092,-0.300736,1,-1.477109,-0.430877,-0.349838,-0.517311,-0.971047,-0.623174,-0.198900,i expect you to die 3 cog in the machine
50468,1272080,PAYDAY 3,2023-09-21,1,0,0,Mostly Negative,38,29458,3.204357,-0.869092,-0.300736,1,-1.470889,-0.430877,-0.349838,-0.517311,-0.971047,-0.623174,-0.198900,payday 3
50469,1402110,Eternights,2023-09-11,1,0,0,Very Positive,89,1128,2.192424,-0.869092,-0.300736,1,-1.462002,-0.430877,-0.349838,-0.517311,-0.971047,-0.623174,-0.198900,eternights
50470,2272250,Forgive Me Father 2,2023-10-19,1,0,0,Very Positive,95,82,0.876911,-0.869092,-0.300736,1,-1.495770,-0.430877,-0.349838,-0.517311,-0.971047,-0.623174,-0.198900,forgive me father 2


# Modelo 1

In [104]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Embedding, Flatten, Dense, Dropout, 
                                   BatchNormalization, Concatenate)
import numpy as np
from sklearn.model_selection import train_test_split

### Separamos los datos en Train 60%, Validation 20% y Test 20%

In [105]:
# Dividir en train/validation/test (60%/20%/20%)
X_temp, X_test, y_temp, y_test, seq_temp, seq_test = train_test_split(
    X, y, padded_sequences, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val, seq_train, seq_val = train_test_split(
    X_temp, y_temp, seq_temp, test_size=0.25, random_state=42
)

print(f"Dimensiones de los datos:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")

Dimensiones de los datos:
X_train: (30282, 14)
X_val: (10095, 14)
X_test: (10095, 14)


### Definimos los parámetros del modelo

In [106]:
vocab_size = min(len(tokenizer.word_index) + 1, 10000)  # +1 para el token de padding
embedding_dim = 32  # Dimensión del espacio de embedding
num_classes = y.shape[1]  # Número de clases (ratings diferentes)

In [107]:
# Input para los títulos
title_input = Input(shape=(max_length,), name='title_input')
# Capa de embedding
embedding_layer = Embedding(vocab_size, embedding_dim, name='embedding')(title_input)
# Aplanar el embedding
flat_embedding = Flatten(name='flatten_embedding')(embedding_layer)

In [108]:
numeric_input = Input(shape=(X.shape[1],), name='numeric_input')
numeric_dense = Dense(64, activation='relu', name='numeric_dense')(numeric_input)
numeric_bn = BatchNormalization(name='numeric_batch_norm')(numeric_dense)

In [109]:
# Concatenar embeddings con características numéricas
concat = Concatenate(name='concatenate')([flat_embedding, numeric_bn])

In [110]:
# Capas densas
dense1 = Dense(128, activation='relu', name='dense1')(concat)
dropout1 = Dropout(0.3, name='dropout1')(dense1)
bn1 = BatchNormalization(name='batch_norm1')(dropout1)

dense2 = Dense(64, activation='relu', name='dense2')(bn1)
dropout2 = Dropout(0.2, name='dropout2')(dense2)
bn2 = BatchNormalization(name='batch_norm2')(dropout2)

In [111]:
# Capa de salida
output = Dense(num_classes, activation='softmax', name='output')(bn2)

In [112]:
# Crear el modelo
model = Model(inputs=[title_input, numeric_input], outputs=output)

In [113]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [114]:
print("\nResumen del modelo:")
model.summary()


Resumen del modelo:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ title_input         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_input       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 10, 32)    │    320,000 │ title_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_dense       │ (None, 64)        │        960 │ numeric_input[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_embedding   │ (None, 320)       │          0 │ embedding[0][0]   │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_batch_norm  │ (None, 64)        │        256 │ numeric_dense[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ flatten_embeddin… │
│ (Concatenate)       │                   │            │ numeric_batch_no… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense1 (Dense)      │ (None, 128)       │     49,280 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout1 (Dropout)  │ (None, 128)       │          0 │ dense1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm1         │ (None, 128)       │        512 │ dropout1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense2 (Dense)      │ (None, 64)        │      8,256 │ batch_norm1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout2 (Dropout)  │ (None, 64)        │          0 │ dense2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm2         │ (None, 64)        │        256 │ dropout2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 9)         │        585 │ batch_norm2[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 380,105 (1.45 MB)

 Trainable params: 379,593 (1.45 MB)

 Non-trainable params: 512 (2.00 KB)

In [115]:
# Forzar el uso de CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Verificar que estamos usando CPU
print("Dispositivos disponibles:", tf.config.list_physical_devices())

Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [116]:
# Entrenamiento del modelo
print("\nEntrenando el modelo...")
history = model.fit(
    # Inputs del modelo
    [seq_train, X_train],  # Lista con las secuencias de títulos y features numéricos
    y_train,               # Labels (ratings)
    
    # Datos de validación
    validation_data=([seq_val, X_val], y_val),
    
    # Parámetros de entrenamiento
    epochs=10,             # Número máximo de epochs
    batch_size=32,        # Tamaño del batch
    
    # Callbacks
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',     # Monitorear pérdida en validación
            patience=3,             # Esperar 3 epochs sin mejora
            restore_best_weights=True  # Restaurar los mejores pesos
        )
    ],
    
    # Mostrar progreso
    verbose=1
)



Entrenando el modelo...
Epoch 1/10


/home/arturo/zProgramming/7th-deep-learning/.venv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['title_input', 'numeric_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(
E0000 00:00:1730506985.168145    5851 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1730506985.224059    5851 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make su

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/arturo/zProgramming/7th-deep-learning/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/arturo/zProgramming/7th-deep-learning/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/arturo/zProgramming/7th-deep-learning/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/arturo/zProgramming/7th-deep-learning/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88,

In [ ]:
a